# Project 1. SQL
https://cs186.gitbook.io/project/assignments/proj1/your-tasks

Date: 2 Mar 2025

## 0. Connecting to PostgreSQL server & data loading

In [117]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

DB_USER = "postgres"
DB_PASS = "221216"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "lahman_baseball"

In [87]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
folder_path = r"..\RESOURCES\Lahman_baseball_statistics_database\lahman_1871-2023_csv"
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

In [118]:
def execute(query):
    return pd.read_sql(text(query), engine)

In [89]:
existing_tables = execute("""
SELECT table_name FROM information_schema.tables 
WHERE table_schema = 'public'
""")["table_name"].tolist()

for file in csv_files:
    table_name = file.replace(".csv", "").lower()
    
    if table_name in existing_tables:
        continue

    file_path = os.path.join(folder_path, file)

    try:
        df = pd.read_csv(file_path, encoding="utf-8", encoding_errors="ignore", 
                         on_bad_lines="skip")  
        df.columns = df.columns.str.lower() 

    except Exception as e:
        print(f"Error loading {file}: {e}")
        continue

    df.to_sql(table_name, engine, if_exists="replace", index=False)

for table in csv_files:
    table_name = table.replace(".csv", "").lower()
    
    col_info = execute(f"""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = '{table_name}'
    """)
    # print(f"Table: {table_name}")

## 1. Data overview

First we would like to have a look at all tables that we loaded onto the server.

In [103]:
execute("""
SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE';
""")

,table_name
0,halloffame
1,homegames
2,managers
3,managershalf
4,parks
5,allstarfull
6,appearances
7,awardsmanagers
8,awardsplayers
9,awardssharemanagers


For the purpose of this project, we are interested in the following main tables:
1. People - Player names, date of birth (DOB), and biographical info
2. Batting - batting statistics
3. Pitching - pitching statistics
4. Fielding - fielding statistics

For more detailed information, see the [README file](https://www.dropbox.com/scl/fi/9i2nhlskvfkqy7mbuqem7/readme2023.txt?rlkey=odnwx7ujztm0z4ob8dmggfcr0&e=2&dl=0).

Now we would like to see the columns in each of the main tables mentioned above: 

In [104]:
table_name = "people"

people_df = execute(f"""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = '{table_name}'
""")

people_df.T # transposed for the sake of space

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
column_name,deathday,weight,height,birthyear,birthmonth,birthday,deathyear,deathmonth,id,namelast,...,finalgame,retroid,playerid,birthcity,birthcountry,birthstate,deathcountry,deathstate,deathcity,namefirst
data_type,double precision,double precision,double precision,double precision,double precision,double precision,double precision,double precision,bigint,text,...,text,text,text,text,text,text,text,text,text,text


In [105]:
table_name = "batting"

batting_df = execute(f"""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = '{table_name}'
""")

batting_df.T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
column_name,g_old,yearid,stint,sh,sf,gidp,g,g_batting,ab,r,...,rbi,sb,cs,bb,so,ibb,hbp,teamid,lgid,playerid
data_type,double precision,bigint,bigint,double precision,double precision,double precision,bigint,double precision,bigint,bigint,...,double precision,double precision,double precision,bigint,double precision,double precision,double precision,text,text,text


In [106]:
table_name = "pitching"

pitching_df = execute(f"""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = '{table_name}'
""")

pitching_df.T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
column_name,gidp,yearid,stint,r,sh,sf,w,l,g,gs,...,era,ibb,wp,hbp,bk,bfp,gf,teamid,lgid,playerid
data_type,double precision,bigint,bigint,bigint,double precision,double precision,bigint,bigint,bigint,bigint,...,double precision,double precision,bigint,double precision,bigint,double precision,bigint,text,text,text


In [107]:
table_name = "fielding"

fielding_df = execute(f"""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = '{table_name}'
""")

fielding_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
column_name,zr,yearid,stint,pb,wp,sb,cs,g,gs,innouts,po,a,e,dp,teamid,lgid,pos,playerid
data_type,double precision,bigint,bigint,double precision,double precision,double precision,double precision,bigint,double precision,double precision,bigint,bigint,double precision,bigint,text,text,text,text


## Task 1. Basics exploration

Difficulty: ★☆☆☆☆

**i. What is the highest earned run average (ERA) recorded in baseball history?**

Since there are too many tables and columns to search through manually, I'll first query for the table that contains a column named "era":

In [108]:
execute("""
SELECT table_name, column_name 
  FROM information_schema.columns 
 WHERE LOWER(column_name) = 'era'
""")

,table_name,column_name
0,pitching,era
1,pitchingpost,era
2,teams,era


In [141]:
question1_1 = execute("""
SELECT MAX(era)  
FROM pitching  
""")
question1_1

,max
0,189.0


**ii. In the `people` table, find the `namefirst`, `namelast` and `birthyear` for all players with weight greater than 300 pounds.**

In [142]:
question1_2 = execute("""
SELECT namefirst, namelast, birthyear
  FROM people
 WHERE weight > 300
""")
question1_2

,namefirst,namelast,birthyear
0,Jumbo,Diaz,1984.0
1,Walter,Young,1980.0
2,John,McSherry,1944.0


**iii. Find the `namefirst`, `namelast` and `birthyear` of all players whose `namefirst` field contains a space. Order the results by `namefirst`, breaking ties with `namelast` both in ascending order.**

In [143]:
question1_3 = execute("""
SELECT namefirst, namelast, birthyear
  FROM people
  WHERE namefirst LIKE '% %'
  ORDER BY namefirst ASC, namelast ASC
""")
question1_3

,namefirst,namelast,birthyear
0,A. J.,Achter,1988.0
1,A. J.,Alexy,1998.0
2,A. J.,Burnett,1977.0
3,A. J.,Cole,1992.0
4,A. J.,Ellis,1981.0
...,...,...,...
149,Vinegar Bend,Mizell,1930.0
150,Wild Bill,Luhrsen,1884.0
151,Wild Bill,Widner,1867.0
152,Wily Mo,Pena,1982.0


**iv. Following the results of part iii, now only include groups with an average height > 70. Again order the results by birthyear in ascending order.**

In [144]:
question1_4 = execute("""
SELECT birthyear, AVG(height)
  FROM people
 GROUP BY birthyear
HAVING AVG(height) > 70
 ORDER BY birthyear;
""")
question1_4

,birthyear,avg
0,1839.0,72.000000
1,1868.0,70.114286
2,1871.0,70.245283
3,1872.0,70.152542
4,1873.0,70.246914
...,...,...
129,1999.0,73.567308
130,2000.0,72.765957
131,2001.0,72.541667
132,2002.0,73.888889


## Task 2. Hall of Fame schools

Difficulty: ★★☆☆☆

**i. Find the `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame in descending order of `yearid`. Break ties on `yearid` by `playerid` (ascending).**

First we would like to have a look at the columns in the table `Hall of Fame`:

In [140]:
execute("""
SELECT column_name 
  FROM information_schema.columns 
 WHERE table_name = 'halloffame'
""")

,column_name
0,votes
1,yearid
2,ballots
3,needed
4,playerid
5,needed_note
6,votedby
7,inducted
8,category


In [163]:
question2_1 = execute("""
SELECT p.namefirst, p.namelast, p.playerid, h.yearid, h.inducted
  FROM people p
  JOIN halloffame h ON p.playerid = h.playerid
 WHERE h.inducted = 'Y'
 ORDER BY h.yearid DESC, p.playerid ASC
""")
question2_1

,namefirst,namelast,playerid,yearid,inducted
0,Adrian,Beltre,beltrad01,2024,Y
1,Todd,Helton,heltoto01,2024,Y
2,Jim,Leyland,leylaji99,2024,Y
3,Joe,Mauer,mauerjo01,2024,Y
4,Fred,McGriff,mcgrifr01,2023,Y
...,...,...,...,...,...
352,Ty,Cobb,cobbty01,1936,Y
353,Walter,Johnson,johnswa01,1936,Y
354,Christy,Mathewson,mathech01,1936,Y
355,Babe,Ruth,ruthba01,1936,Y


**ii. Find the people who were successfully inducted into the Hall of Fame and played in college at a school located in the state of California. For each person, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` in descending order of `yearid`. Break ties on `yearid` by `schoolid`, `playerid` (ascending). For this question, `yearid` refers to the year of induction into the Hall of Fame.**

Note: a player may appear in the results multiple times (once per year in a college in California).

In [160]:
display(execute("""
SELECT column_name 
  FROM information_schema.columns 
 WHERE table_name = 'collegeplaying'
"""))

execute("""
SELECT * FROM collegeplaying 
 WHERE schoolid LIKE '%cali%'
""")

,column_name
0,yearid
1,playerid
2,schoolid


,playerid,schoolid,yearid
0,albrija01,california,1941
1,albrija01,california,1942
2,albrija01,california,1943
3,bakerjo01,california,2000
4,bakerjo01,california,2001
...,...,...,...
166,werlebi01,california,1942
167,zuberjo01,california,1989
168,zuberjo01,california,1990
169,zuberjo01,california,1991


In [167]:
question2_2 = execute("""
SELECT p.namefirst, p.namelast, p.playerid, 
       h.yearid, c.schoolid
  FROM people p
  JOIN halloffame h ON p.playerid = h.playerid
  JOIN collegeplaying c ON p.playerid = c.playerid
 WHERE h.inducted = 'Y' AND
       c.schoolid = 'california'
 ORDER BY h.yearid DESC, c.schoolid ASC, p.playerid ASC
""")
question2_2

,namefirst,namelast,playerid,yearid,schoolid


No player from a school in California fits the description.

**iii. Find the `playerid`, `namefirst`, `namelast` and `schoolid` of all people who were successfully inducted into the Hall of Fame -- whether or not they played in college. Return people in descending order of `playerid`. Break ties on `playerid` by `schoolid` (ascending).**

Note: `schoolid` should be NULL if they did not play in college.

In [171]:
question2_3 = execute("""
SELECT p.playerid, p.namefirst, p.namelast, c.schoolid
  FROM people p 
  LEFT JOIN collegeplaying c ON c.playerid = p.playerid
  JOIN halloffame h ON h.playerid = p.playerid
 WHERE h.inducted = 'Y'
 ORDER BY p.playerid DESC, c.schoolid ASC
""")
question2_3

,playerid,namefirst,namelast,schoolid
0,yountro01,Robin,Yount,None
1,youngro01,Ross,Youngs,None
2,youngcy01,Cy,Young,None
3,yawketo99,Tom,Yawkey,None
4,yastrca01,Carl,Yastrzemski,None
...,...,...,...,...
432,alstowa01,Walter,Alston,miamioh
433,alstowa01,Walter,Alston,miamioh
434,alomaro01,Roberto,Alomar,None
435,alexape01,Pete,Alexander,None


## Task 3. SaberMetrics

Difficulty: ★★★★☆

**i. Find the `playerid`, `namefirst`, `namelast`, `yearid` and single-year `slg` (Slugging Percentage) of the players with the 10 best annual Slugging Percentage recorded over all time. A player can appear multiple times in the output.**

For example, if Babe Ruth’s `slg` in 2000 and 2001 both landed in the top 10 best annual Slugging Percentage of all time, then we should include Babe Ruth twice in the output. For statistical significance, only include players with more than 50 at-bats in the season. Order the results by `slg` descending, and break ties by `yearid`, `playerid` (ascending).

* Baseball note: Slugging Percentage is not provided in the database; it is computed according to a simple formula you can calculate from the data in the database.
* SQL note: You should compute `slg` properly as a floating point number---you'll need to figure out how to convince SQL to do this!
* Data set note: The online documentation batting mentions two columns 2B and 3B. On your local copy of the data set these have been renamed `H2B` and `H3B` respectively (columns starting with numbers are tedious to write queries on).
* Data set note: The column `H` of the batting table represents all hits = (# singles) + (# doubles) + (# triples) + (# home runs), not just (# singles) so you’ll need to account for some double-counting
* If a player played on multiple teams during the same season (for example anderma02 in 2006) treat their time on each team separately for this calculation.

Slugging percentage (SLG) is calculated as:

$$
SLG = \frac{(1B + 2 \times 2B + 3 \times 3B + 4 \times HR)}{AB}
$$

Where:
- **1B** = Singles = `H - 2B - 3B - HR`
- **2B** = Doubles (`2B`)
- **3B** = Triples (`3B`)
- **HR** = Home Runs (`HR`)
- **AB** = At Bats (`AB`)

In [184]:
question3_1 = execute("""
SELECT p.playerid, p.namefirst, p.namelast, b.yearid, 
       (CAST((b.h - b."2b" - b."3b" - b.hr + 2*b."2b" 
              + 3*b."3b" + 4*b.hr) AS FLOAT) 
        / NULLIF(b.ab, 0)) AS slg
  FROM batting b
  JOIN people p ON p.playerid = b.playerid
 WHERE b.ab > 50
 ORDER BY slg DESC, b.yearid ASC, p.playerid ASC
 LIMIT 10
""")
question3_1

,playerid,namefirst,namelast,yearid,slg
0,spencsh01,Shane,Spencer,1998,0.910448
1,willite01,Ted,Williams,1953,0.901099
2,bondsba01,Barry,Bonds,2001,0.863445
3,ruthba01,Babe,Ruth,1920,0.849015
4,ruthba01,Babe,Ruth,1921,0.846296
5,bakerje03,Jeff,Baker,2006,0.824561
6,anderma02,Marlon,Anderson,2006,0.812500
7,bondsba01,Barry,Bonds,2004,0.812332
8,bondsba01,Barry,Bonds,2002,0.799007
9,ruthba01,Babe,Ruth,1927,0.772222


**ii. Following the results from Part i, find the `playerid`, `namefirst`, `namelast` and `lslg` (Lifetime Slugging Percentage) for the players with the top 10 Lifetime Slugging Percentage.**

Lifetime Slugging Percentage (LSLG) uses the same formula as Slugging Percentage (SLG), but it uses the number of singles, doubles, triples, home runs, and at bats each player has over their entire career, rather than just over a single season.

- Note that the database only gives batting information broken down by year; you will need to convert to total information across all time (from the earliest date recorded up to the last date recorded) to compute lslg. Order the results by lslg (descending) and break ties by playerid (ascending)
- Note: Make sure that you only include players with more than 50 at-bats across their lifetime.

In [186]:
question3_2 = execute("""
SELECT p.playerid, p.namefirst, p.namelast, 
       (CAST((SUM(b.h) - SUM(b."2b") - SUM(b."3b") - SUM(b.hr) 
              + 2*SUM(b."2b") + 3*SUM(b."3b") + 4*SUM(b.hr)) AS FLOAT) 
        / NULLIF(SUM(b.ab), 0)) AS lslg
  FROM batting b
  JOIN people p ON p.playerid = b.playerid
 GROUP BY p.playerid, p.namefirst, p.namelast
 HAVING SUM(b.ab) > 50
 ORDER BY lslg DESC, p.playerid ASC
 LIMIT 10
""")
question3_2

,playerid,namefirst,namelast,lslg
0,ruthba01,Babe,Ruth,0.689807
1,carteev01,Evan,Carter,0.645161
2,willite01,Ted,Williams,0.633792
3,gehrilo01,Lou,Gehrig,0.632421
4,foxxji01,Jimmie,Foxx,0.609294
5,bondsba01,Barry,Bonds,0.606885
6,greenha01,Hank,Greenberg,0.605045
7,schneda03,Davis,Schneider,0.603448
8,bassjo01,John,Bass,0.600000
9,mcgwima01,Mark,McGwire,0.588169


**iii. Find the `namefirst`, `namelast` and Lifetime Slugging Percentage (`lslg`) of batters whose lifetime slugging percentage is higher than that of San Francisco favorite Willie Mays.**

You may include Willie Mays' playerid in your query (mayswi01), but you may not include his slugging percentage -- you should calculate that as part of the query. (Test your query by replacing mayswi01 with the playerid of another player -- it should work for that player as well!)
- Note: Make sure that you still only include players with more than 50 at-bats across their lifetime.

"First we need to find Willie Mays' lstg using WITH (a common table expression for creating views) then find all the other players greater than the calculated lslg."

That was my initial approach until I realised that I would have to type the very long formula twice, once for Mays and one for the rest of the players. 

Thus I decided to use 2 WITH clauses, one for the formula, then one for Mays in which I used the previously defined view. 




In [ ]:
question3_3 = execute("""

WITH lstgs AS (
    SELECT 
        p.playerid, p.namefirst, p.namelast,
        (CAST((SUM(b.h) - SUM(b."2b") - SUM(b."3b") - SUM(b.hr)  
              + 2*SUM(b."2b") + 3*SUM(b."3b") + 4*SUM(b.hr)) AS FLOAT)  
        / NULLIF(SUM(b.ab), 0)) AS lslg, 
        SUM(b.ab) AS tab
    FROM people p
    JOIN batting b ON b.playerid = p.playerid
    GROUP BY p.playerid, p.namefirst, p.namelast
),
willie_mays_lslg AS (
    SELECT lslg AS mays_lslg
    FROM lstgs
    WHERE playerid = 'mayswi01'
)

SELECT l.namefirst, l.namelast, l.lslg
  FROM lstgs l
  JOIN willie_mays_lslg w ON l.lslg > w.mays_lslg
   AND l.tab > 50
 ORDER BY l.lslg DESC;

""")
question3_3

,namefirst,namelast,lslg
0,Babe,Ruth,0.689807
1,Evan,Carter,0.645161
2,Ted,Williams,0.633792
3,Lou,Gehrig,0.632421
4,Jimmie,Foxx,0.609294
5,Barry,Bonds,0.606885
6,Hank,Greenberg,0.605045
7,Davis,Schneider,0.603448
8,John,Bass,0.600000
9,Mark,McGwire,0.588169


## Task 4. Salaries

Difficulty: ★★★★★

**i. Find the `yearid`, min, max and average of all player salaries for each year recorded, ordered by `yearid` in ascending order.**

In [239]:
execute("""
SELECT column_name
  FROM information_schema.columns
 WHERE table_name = 'salaries'
""")

,column_name
0,yearid
1,salary
2,teamid
3,lgid
4,playerid


In [237]:
question4_1 = execute("""
SELECT yearid, MIN(salary), 
       MAX(salary), CAST(AVG(salary) AS INT)
  FROM salaries
 GROUP BY yearid
 ORDER BY yearid ASC
""")
question4_1

,yearid,min,max,avg
0,1985,60000,2130300,476299
1,1986,60000,2800000,417147
2,1987,62500,2127333,434729
3,1988,62500,2340000,453171
4,1989,62500,2766667,506323
5,1990,100000,3200000,511974
6,1991,100000,3800000,894961
7,1992,109000,6100000,1047521
8,1993,0,6200000,976967
9,1994,50000,6300000,1049589


**ii. For salaries in 2016, compute a histogram. Divide the salary range into 10 equal bins from min to max, with binids 0 through 9, and count the salaries in each bin. Return the binid, low and high boundaries for each bin, as well as the number of salaries in each bin, with results sorted from smallest bin to largest.**

- Note: binid 0 corresponds to the lowest salaries, and binid 9 corresponds to the highest. The ranges are left-inclusive (i.e. [low, high)) -- so the high value is excluded. For example, if bin 2 has a high value of 100000, salaries of 100000 belong in bin 3, and bin 3 should have a low value of 100000.

- Note: The high value for bin 9 may be inclusive).

My approach:
1. Find min and max value
2. Find bin width
3. Determine bin for each value
4. Group by binid and count
5. Find the max and min salary of each bin

In [ ]:
execute ("""
WITH info AS (
    SELECT MIN(salary) AS min_salary, 
           MAX(salary) AS max_salary, 
           CAST(AVG(salary) AS INT) AS avg_salary,
           (MAX(salary) - MIN(salary)) / 10 AS bin_width
    FROM salaries
    WHERE yearid = 2016
),
binned AS (
    SELECT 
        LEAST(FLOOR((s.salary - i.min_salary) / i.bin_width), 9) AS binid,
        s.salary
    FROM salaries s
    JOIN info i ON 1=1
    WHERE s.yearid = 2016
)

SELECT 
    b.binid, 
    MIN(b.salary) AS low, 
    MAX(b.salary) AS high, 
    COUNT(*) AS salary_count
FROM binned b
GROUP BY b.binid  
ORDER BY b.binid;
""")

,binid,low,high,salary_count
0,0.0,507500,3750000,558
1,1.0,3800000,7000000,114
2,2.0,7150000,10000000,60
3,3.0,10400000,13500000,44
4,4.0,13666667,16329674,24
5,5.0,16800000,20000000,19
6,6.0,20125000,23125000,21
7,7.0,24000000,25857143,7
8,8.0,27328046,28000000,3
9,9.0,30000000,33000000,3
